In [64]:
import jupyter_manim
import numpy as np
from manim import *
import heapq

config.media_width = "100%"
config.verbosity = "WARNING"

In [134]:
# This example constructs a simple graph of 2 dimensional vectors, connected to their 5 nearest neighbors
# and then performs a breadth first search to find the nearest neighbor to a target vector
# distance is measured using Euclidean distance
# ensure to seed the random numbers, we will end up doing other visualizations with vectors later
np.random.seed(42)
        # Create a set of random 2D points
num_points = 100
points = np.random.rand(num_points, 2) * 6 - 3  # Scale to fit in the scene
# make points 3d by adding a zero z-coordinate
points = np.hstack((points, np.zeros((num_points, 1))))
target = np.array([0.5, 2.4, 0])
# find the furthest point to demonstrate worst case search
distances = np.linalg.norm(points - target, axis=1)
start_index = np.argmax(distances)
# Connect each point to its 5 nearest neighbors
edges = []
idx_to_neighbors = {}
for i in range(num_points):
    distances = np.linalg.norm(points - points[i], axis=1)
    nearest_indices = np.argsort(distances)[1:6]  # Exclude self (index 0)
    for j in nearest_indices:
        edges.append((i, j))
        if i not in idx_to_neighbors:
            idx_to_neighbors[i] = []
        if j not in idx_to_neighbors:
            idx_to_neighbors[j] = []
        # edges are undirected
        idx_to_neighbors[i].append(j)
        idx_to_neighbors[j].append(i)

# we purposefully ignore the z-coordinate in distance calculations
def calculate_distance(point, target):
    return np.linalg.norm(point[:2] - target[:2])

In [66]:
class GraphSearchExample(ThreeDScene):
    # pass in the graph data in ctor
    def __init__(self, points, edges, idx_to_neighbors, start_index, **kwargs):
        self.graph_data = {
            "points": points,
            "edges": edges,
            "idx_to_neighbors": idx_to_neighbors
        }
        self.start_index = start_index
        super().__init__(**kwargs)
    
    def construct(self):
        
        num_points = len(self.graph_data["points"])
        points = self.graph_data["points"]
        edges = self.graph_data["edges"]
        idx_to_neighbors = self.graph_data["idx_to_neighbors"]
        # Create a graph using the points as vertices
        graph = Graph(
            vertices=[i for i in range(num_points)],
            edges=edges,
            # remember to convert 2D points to 3D by adding a zero z-coordinate
            layout={i: points[i] for i in range(num_points)},
            vertex_config={"radius": 0.075, "color": BLUE},
            edge_config={"color": GREY},
        )


        self.play(Create(graph))
        self.wait(1)

        # Define the target point
        target_point = np.array([0.5, 2.4, 0])
        target_dot = Dot(point=target_point, color=RED)
        self.play(FadeIn(target_dot))
        self.wait(1)

        # Perform a depth-first search to find the nearest neighbor to the target point, highlighting the search process by changing the color of the vertices and edges as they are explored to green
        score = calculate_distance(points[self.start_index], target_point)
        # we need a min-heap to explore the lowest score first
        stack = [(score, self.start_index)]  # (score, vertex)
        heapq.heapify(stack)
        visited = set()
        found_vertex = None
        num_hops = 0
        min_score = float('inf')
        while stack:
            score, current_vertex = heapq.heappop(stack)
            if current_vertex in visited:
                continue
            visited.add(current_vertex)

            # Check if this vertex is the closest to the target point
            if score < min_score:
                num_hops += 1
                score_text = Text(f"Visiting {current_vertex}, Score: {score:.2f}", font_size=24).to_corner(UL)
                self.play(AnimationGroup(Write(score_text), graph[current_vertex].animate.set_color(YELLOW)))
                previous = found_vertex
                found_vertex = current_vertex
                min_score = score
                animations = []
                if previous is not None:
                    animations.append(graph[previous].animate.set_color(WHITE))
                animations.append(graph[current_vertex].animate.set_color(GREEN))
                animations.append(FadeOut(score_text))
                self.play(*animations)
                for neighbor in idx_to_neighbors.get(current_vertex, []):
                    neighbor_score = calculate_distance(points[neighbor], target_point)
                    print(f"{current_vertex} Adding Neighbor {neighbor}, Score: {neighbor_score:.2f} Min Score: {min_score:.2f}")
                    if neighbor not in visited and neighbor_score < min_score:
                        heapq.heappush(stack, (neighbor_score, neighbor))


        # Highlight the found vertex
        self.play(Text(f"Found Nearest Neighbor in {num_hops} hops!", font_size=36).to_edge(UP).animate.set_color(GREEN))
        self.wait(2)

In [67]:
%%manim -qm NearestNeighborExample

class NearestNeighborExample(GraphSearchExample):
    def __init__(self, **kwargs):
        super().__init__(points, edges, idx_to_neighbors, start_index, **kwargs)

Manim Community v0.18.1

49 Adding Neighbor 2, Score: 5.15 Min Score: 5.81
49 Adding Neighbor 28, Score: 5.16 Min Score: 5.81
49 Adding Neighbor 28, Score: 5.16 Min Score: 5.81
49 Adding Neighbor 2, Score: 5.15 Min Score: 5.81
49 Adding Neighbor 74, Score: 4.91 Min Score: 5.81
49 Adding Neighbor 7, Score: 4.93 Min Score: 5.81
49 Adding Neighbor 54, Score: 5.38 Min Score: 5.81


74 Adding Neighbor 28, Score: 5.16 Min Score: 4.91
74 Adding Neighbor 29, Score: 4.72 Min Score: 4.91
74 Adding Neighbor 49, Score: 5.81 Min Score: 4.91
74 Adding Neighbor 66, Score: 4.37 Min Score: 4.91
74 Adding Neighbor 29, Score: 4.72 Min Score: 4.91
74 Adding Neighbor 66, Score: 4.37 Min Score: 4.91
74 Adding Neighbor 28, Score: 5.16 Min Score: 4.91
74 Adding Neighbor 2, Score: 5.15 Min Score: 4.91
74 Adding Neighbor 7, Score: 4.93 Min Score: 4.91


66 Adding Neighbor 29, Score: 4.72 Min Score: 4.37
66 Adding Neighbor 29, Score: 4.72 Min Score: 4.37
66 Adding Neighbor 74, Score: 4.91 Min Score: 4.37
66 Adding Neighbor 62, Score: 3.55 Min Score: 4.37
66 Adding Neighbor 28, Score: 5.16 Min Score: 4.37
66 Adding Neighbor 76, Score: 3.60 Min Score: 4.37
66 Adding Neighbor 74, Score: 4.91 Min Score: 4.37


62 Adding Neighbor 8, Score: 2.81 Min Score: 3.55
62 Adding Neighbor 11, Score: 3.65 Min Score: 3.55
62 Adding Neighbor 13, Score: 3.26 Min Score: 3.55
62 Adding Neighbor 20, Score: 3.68 Min Score: 3.55
62 Adding Neighbor 23, Score: 2.80 Min Score: 3.55
62 Adding Neighbor 42, Score: 3.82 Min Score: 3.55
62 Adding Neighbor 51, Score: 2.85 Min Score: 3.55
62 Adding Neighbor 11, Score: 3.65 Min Score: 3.55
62 Adding Neighbor 13, Score: 3.26 Min Score: 3.55
62 Adding Neighbor 76, Score: 3.60 Min Score: 3.55
62 Adding Neighbor 51, Score: 2.85 Min Score: 3.55
62 Adding Neighbor 8, Score: 2.81 Min Score: 3.55
62 Adding Neighbor 66, Score: 4.37 Min Score: 3.55
62 Adding Neighbor 76, Score: 3.60 Min Score: 3.55
62 Adding Neighbor 97, Score: 3.62 Min Score: 3.55


23 Adding Neighbor 8, Score: 2.81 Min Score: 2.80
23 Adding Neighbor 8, Score: 2.81 Min Score: 2.80
23 Adding Neighbor 51, Score: 2.85 Min Score: 2.80
23 Adding Neighbor 32, Score: 2.81 Min Score: 2.80
23 Adding Neighbor 13, Score: 3.26 Min Score: 2.80
23 Adding Neighbor 62, Score: 3.55 Min Score: 2.80
23 Adding Neighbor 32, Score: 2.81 Min Score: 2.80
23 Adding Neighbor 51, Score: 2.85 Min Score: 2.80
23 Adding Neighbor 80, Score: 2.06 Min Score: 2.80


80 Adding Neighbor 12, Score: 1.03 Min Score: 2.06
80 Adding Neighbor 22, Score: 2.41 Min Score: 2.06
80 Adding Neighbor 53, Score: 1.35 Min Score: 2.06
80 Adding Neighbor 68, Score: 1.24 Min Score: 2.06
80 Adding Neighbor 79, Score: 2.32 Min Score: 2.06
80 Adding Neighbor 22, Score: 2.41 Min Score: 2.06
80 Adding Neighbor 92, Score: 2.43 Min Score: 2.06
80 Adding Neighbor 32, Score: 2.81 Min Score: 2.06
80 Adding Neighbor 8, Score: 2.81 Min Score: 2.06
80 Adding Neighbor 23, Score: 2.80 Min Score: 2.06
80 Adding Neighbor 92, Score: 2.43 Min Score: 2.06


12 Adding Neighbor 0, Score: 1.29 Min Score: 1.03
12 Adding Neighbor 4, Score: 1.16 Min Score: 1.03
12 Adding Neighbor 53, Score: 1.35 Min Score: 1.03
12 Adding Neighbor 68, Score: 1.24 Min Score: 1.03
12 Adding Neighbor 4, Score: 1.16 Min Score: 1.03
12 Adding Neighbor 80, Score: 2.06 Min Score: 1.03
12 Adding Neighbor 0, Score: 1.29 Min Score: 1.03
12 Adding Neighbor 27, Score: 0.16 Min Score: 1.03
12 Adding Neighbor 53, Score: 1.35 Min Score: 1.03
12 Adding Neighbor 68, Score: 1.24 Min Score: 1.03
12 Adding Neighbor 69, Score: 1.39 Min Score: 1.03


27 Adding Neighbor 0, Score: 1.29 Min Score: 0.16
27 Adding Neighbor 57, Score: 0.35 Min Score: 0.16
27 Adding Neighbor 98, Score: 0.86 Min Score: 0.16
27 Adding Neighbor 78, Score: 0.99 Min Score: 0.16
27 Adding Neighbor 12, Score: 1.03 Min Score: 0.16
27 Adding Neighbor 60, Score: 1.34 Min Score: 0.16
27 Adding Neighbor 57, Score: 0.35 Min Score: 0.16
27 Adding Neighbor 69, Score: 1.39 Min Score: 0.16
27 Adding Neighbor 98, Score: 0.86 Min Score: 0.16


In [60]:
%%manim -qm RandomNeighborExample
# now lets adjust the graph to adjust the edges so that 1% of the points have a couple random connections to other points
#  in addition to their nearest neighbors 
# --- IGNORE ---
np.random.seed(42)
for i in range(num_points):
    if np.random.rand() < 0.05:
        # now add 2 random edges to other points
        # to ensure we don't connect to self or existing neighbors,
        possible_indices = list(set(range(num_points)) - {i} - set(idx_to_neighbors.get(i, [])))
        if len(possible_indices) >= 2:
            random_indices = np.random.choice(possible_indices, 2, replace=False)
        for j in random_indices:
            edges.append((i, j))
            # edges are undirected
            idx_to_neighbors[i].append(j)
            idx_to_neighbors[j].append(i)
class RandomNeighborExample(GraphSearchExample):
    def __init__(self, **kwargs):
        super().__init__(points, edges, idx_to_neighbors, start_index, **kwargs)

Manim Community v0.18.1

49 Adding Neighbor 2, Score: 5.15 Min Score: 5.81
49 Adding Neighbor 28, Score: 5.16 Min Score: 5.81
49 Adding Neighbor 28, Score: 5.16 Min Score: 5.81
49 Adding Neighbor 2, Score: 5.15 Min Score: 5.81
49 Adding Neighbor 74, Score: 4.91 Min Score: 5.81
49 Adding Neighbor 7, Score: 4.93 Min Score: 5.81
49 Adding Neighbor 54, Score: 5.38 Min Score: 5.81
49 Adding Neighbor 80, Score: 2.06 Min Score: 5.81


80 Adding Neighbor 12, Score: 1.03 Min Score: 2.06
80 Adding Neighbor 22, Score: 2.41 Min Score: 2.06
80 Adding Neighbor 53, Score: 1.35 Min Score: 2.06
80 Adding Neighbor 68, Score: 1.24 Min Score: 2.06
80 Adding Neighbor 79, Score: 2.32 Min Score: 2.06
80 Adding Neighbor 22, Score: 2.41 Min Score: 2.06
80 Adding Neighbor 92, Score: 2.43 Min Score: 2.06
80 Adding Neighbor 32, Score: 2.81 Min Score: 2.06
80 Adding Neighbor 8, Score: 2.81 Min Score: 2.06
80 Adding Neighbor 23, Score: 2.80 Min Score: 2.06
80 Adding Neighbor 92, Score: 2.43 Min Score: 2.06
80 Adding Neighbor 49, Score: 5.81 Min Score: 2.06
80 Adding Neighbor 84, Score: 3.75 Min Score: 2.06


12 Adding Neighbor 0, Score: 1.29 Min Score: 1.03
12 Adding Neighbor 4, Score: 1.16 Min Score: 1.03
12 Adding Neighbor 53, Score: 1.35 Min Score: 1.03
12 Adding Neighbor 68, Score: 1.24 Min Score: 1.03
12 Adding Neighbor 4, Score: 1.16 Min Score: 1.03
12 Adding Neighbor 80, Score: 2.06 Min Score: 1.03
12 Adding Neighbor 0, Score: 1.29 Min Score: 1.03
12 Adding Neighbor 27, Score: 0.16 Min Score: 1.03
12 Adding Neighbor 53, Score: 1.35 Min Score: 1.03
12 Adding Neighbor 68, Score: 1.24 Min Score: 1.03
12 Adding Neighbor 69, Score: 1.39 Min Score: 1.03


27 Adding Neighbor 0, Score: 1.29 Min Score: 0.16
27 Adding Neighbor 57, Score: 0.35 Min Score: 0.16
27 Adding Neighbor 98, Score: 0.86 Min Score: 0.16
27 Adding Neighbor 78, Score: 0.99 Min Score: 0.16
27 Adding Neighbor 12, Score: 1.03 Min Score: 0.16
27 Adding Neighbor 60, Score: 1.34 Min Score: 0.16
27 Adding Neighbor 57, Score: 0.35 Min Score: 0.16
27 Adding Neighbor 69, Score: 1.39 Min Score: 0.16
27 Adding Neighbor 98, Score: 0.86 Min Score: 0.16


In [ ]:
%%manim -qm PurelyRandomNeighborExample
# now lets see what a purely random neighbor example looks like
#  where each point has 5 random connections to other points

np.random.seed(0)
random_idx_to_neighbors = {}
random_edges = []
for i in range(num_points):
    random_neighbors = np.random.choice(num_points, 6, replace=False)
    # make sure we don't connect to self
    random_neighbors = [j for j in random_neighbors if j != i]
    random_neighbors = random_neighbors[:5]  # limit to 5 neighbors
    random_idx_to_neighbors[i] = []
    for j in random_neighbors:
        random_edges.append((i, j))
        random_idx_to_neighbors[i].append(j)
        if j not in random_idx_to_neighbors:
            random_idx_to_neighbors[j] = []
        random_idx_to_neighbors[j].append(i)  # undirected edge

class PurelyRandomNeighborExample(GraphSearchExample):
    def __init__(self, **kwargs):
        super().__init__(points, random_edges, random_idx_to_neighbors, start_index, **kwargs)

Manim Community v0.18.1

49 Adding Neighbor 64, Score: 4.17 Min Score: 5.81
49 Adding Neighbor 81, Score: 2.21 Min Score: 5.81
49 Adding Neighbor 1, Score: 2.02 Min Score: 5.81
49 Adding Neighbor 66, Score: 4.37 Min Score: 5.81
49 Adding Neighbor 68, Score: 1.24 Min Score: 5.81
49 Adding Neighbor 59, Score: 2.85 Min Score: 5.81
49 Adding Neighbor 86, Score: 4.06 Min Score: 5.81


68 Adding Neighbor 74, Score: 4.91 Min Score: 1.24
68 Adding Neighbor 23, Score: 2.80 Min Score: 1.24
68 Adding Neighbor 59, Score: 2.85 Min Score: 1.24
68 Adding Neighbor 39, Score: 4.89 Min Score: 1.24
68 Adding Neighbor 98, Score: 0.86 Min Score: 1.24
68 Adding Neighbor 69, Score: 1.39 Min Score: 1.24
68 Adding Neighbor 76, Score: 3.60 Min Score: 1.24
68 Adding Neighbor 91, Score: 2.05 Min Score: 1.24


98 Adding Neighbor 44, Score: 3.15 Min Score: 0.86
98 Adding Neighbor 81, Score: 2.21 Min Score: 0.86
98 Adding Neighbor 20, Score: 3.68 Min Score: 0.86
98 Adding Neighbor 55, Score: 4.77 Min Score: 0.86
98 Adding Neighbor 45, Score: 3.00 Min Score: 0.86


In [132]:
%%manim -qm ThreeDGraphSearchExample
num_points = 100
edges = []
idx_to_neighbors = {}
for i in range(num_points):
    distances = np.linalg.norm(points - points[i], axis=1)
    nearest_indices = np.argsort(distances)[1:6]  # Exclude self (index 0)
    for j in nearest_indices:
        edges.append((i, j))
        if i not in idx_to_neighbors:
            idx_to_neighbors[i] = []
        if j not in idx_to_neighbors:
            idx_to_neighbors[j] = []
        # edges are undirected
        idx_to_neighbors[i].append(j)
        idx_to_neighbors[j].append(i)

# here we cheat, and add a second layer of connections that are on the higher z plane
# this is to illustrate HNSW style graphs
second_layer_vertices = []
second_layer_edges = []
chance_of_higher_layer = 0.05
np.random.seed(42)
second_layer_offset = num_points  # offset for second layer indices

for i in range(num_points):
    if np.random.rand() < chance_of_higher_layer:
        # Connect to a random point in the higher layer
        second_layer_vertices.append(i)  # z=2 for higher layer
# create edges between the vectors on the higher layer, using the nearest neighbor approach again
second_layer_points = points[second_layer_vertices] + np.array([0, 0, 2])
second_layer_idx_to_neighbors = {}
# i just needs to be the index in the second layer + the offset
for i, point in enumerate(second_layer_points):
    distances = np.linalg.norm(second_layer_points - point, axis=1)
    nearest_indices = np.argsort(distances)[1:3]  # Exclude self (index 0)
    for j in nearest_indices:
        neighbor_idx = j + second_layer_offset
        second_layer_edges.append((i + second_layer_offset, neighbor_idx))
        # edges are undirected
        if i not in second_layer_idx_to_neighbors:
            second_layer_idx_to_neighbors[i + second_layer_offset] = []
        if neighbor_idx not in second_layer_idx_to_neighbors:
            second_layer_idx_to_neighbors[neighbor_idx] = []
        second_layer_idx_to_neighbors[i + second_layer_offset].append(neighbor_idx)
        second_layer_idx_to_neighbors[neighbor_idx].append(i + second_layer_offset)
print(f"Second layer has {len(second_layer_vertices)} points")
print(f"Total edges before adding second layer: {second_layer_edges}")
edges.extend(second_layer_edges)
threed_points = np.vstack((points, second_layer_points))
num_points = len(threed_points)
# make sure we only score the first two dimensions
# find the furthest point on the second layer to demonstrate worst case search
distances = np.linalg.norm(second_layer_points[:, :2] - target[:2], axis=1)
start_index = np.argmax(distances) + second_layer_offset

print(f"3D Graph has {num_points} points, starting at {start_index}")

class ThreeDGraphSearchExample(ThreeDScene):

    def construct(self):
        
        # Create a graph using the points as vertices
        graph = Graph(
            vertices=[i for i in range(num_points)],
            edges=edges,
            layout={i: threed_points[i] for i in range(num_points)},
            vertex_config={"radius": 0.075, "color": BLUE},
            edge_config={"color": GREY},
        )

        self.play(Create(graph))
        self.wait(0.5)
        # pan down the camera smoothly and stop at the new position
        self.move_camera(phi=65 * DEGREES, theta=-45 * DEGREES, run_time=3)
        #self.set_camera_orientation(phi=65 * DEGREES, theta=-45 * DEGREES)
        self.wait(1)


        # Define the target point
        target_point = np.array([0.5, 2.4, 0])
        target_dot = Dot(point=target_point, color=RED)
        self.play(FadeIn(target_dot))
        self.wait(1)
        print(f"second_layer_offset: {second_layer_offset}")
        print(f"second_layer_vertices: {second_layer_vertices}")
        print(f"len(second_layer_vertices): {len(second_layer_vertices)}")
    
        # first explore the second layer of the graph
        score = calculate_distance(threed_points[start_index], target_point)
        stack = [(score, start_index)]  # (score, vertex)
        heapq.heapify(stack)
        visited = set()
        second_layer_vertex = None
        num_hops = 0
        min_score = float('inf')
        while stack:
            score, current_vertex = heapq.heappop(stack)
            if current_vertex in visited:
                continue
            visited.add(current_vertex)

            # Check if this vertex is the closest to the target point
            if score < min_score:
                num_hops += 1
                # make sure text is oriented to camera
                print(f"Visiting {current_vertex}, Score: {score:.2f}")
                score_text = Text(f"Visiting {second_layer_vertices[current_vertex - second_layer_offset]}, Score: {score:.2f}", font_size=24)
                score_text.to_corner(UL)
                self.add_fixed_in_frame_mobjects(score_text)
                self.remove(score_text)
                self.play(AnimationGroup(Write(score_text), graph[current_vertex].animate.set_color(YELLOW)))
                self.wait(0.25)
                previous = second_layer_vertex
                second_layer_vertex = current_vertex
                min_score = score
                animations = []
                if previous is not None:
                    animations.append(graph[previous].animate.set_color(WHITE))
                animations.append(graph[current_vertex].animate.set_color(GREEN))
                animations.append(FadeOut(score_text))
                self.play(*animations)
                print(f"getting neighbors for {current_vertex} in second layer: {second_layer_idx_to_neighbors}")
                for neighbor in second_layer_idx_to_neighbors.get(current_vertex, []):
                    neighbor = neighbor
                    neighbor_score = calculate_distance(threed_points[neighbor], target_point)
                    print(f"{current_vertex} Adding Neighbor {neighbor}, Score: {neighbor_score:.2f} Min Score: {min_score:.2f}")
                    if neighbor not in visited and neighbor_score < min_score:
                        heapq.heappush(stack, (neighbor_score, neighbor))
        ############
        # now we can start on the full graph search
        txt = Text(f"Second Layer Nearest Neighbor in {num_hops} hops!", font_size=36)
        # orient the text to the camera
        self.add_fixed_in_frame_mobjects(txt)
        txt.to_edge(UP)
        txt.set_color(GREEN)
        self.play(Write(txt))
        self.wait(2)
        self.play(FadeOut(txt))
        # move camera to immeidately above the graph
        self.move_camera(phi=DEGREES, theta=-90 * DEGREES, run_time=3)
        self.wait(1)
        # now make the second layer vertices and edges less prominent
        to_fade = []
        for v in range(100, num_points):
            to_fade.append(FadeOut(graph[v]))
        # now handle all second layer edges
        faded_edges = set()
        for edge in second_layer_edges:
            to_fade.append(FadeOut(graph.edges[edge]))
            #if edge not in faded_edges and (edge[1], edge[0]) not in faded_edges:
            #    faded_edges.add(edge)
        self.play(*to_fade)

        print(f"init full graph search from {second_layer_vertex}, score {min_score:.2f}")

        # Debug information
        print(f"second_layer_vertex: {second_layer_vertex} adjusted: {second_layer_vertex - second_layer_offset}")
        print(f"related first layer: {second_layer_vertices[second_layer_vertex - second_layer_offset]}")

        # Perform a depth-first search to find the nearest neighbor to the target point, highlighting the search process by changing the color of the vertices and edges as they are explored to green
        # we need a min-heap to explore the lowest score first
        stack = []
        heapq.heapify(stack)
        visited = set()
        # transform second_layer_vertex to the lower layer, knowing the offset
        # second_layer_vertex is the index in the full 3D graph, we need to find the corresponding original vertex
        original_vertex_idx = second_layer_vertices[second_layer_vertex - second_layer_offset]  # this gives us the index in second_layer_vertices list
        print(f"original_vertex_idx: {original_vertex_idx}")
        score = min_score 
        stack = [(score, original_vertex_idx)]  # (score, vertex)
        print(f"Starting full graph search from {original_vertex_idx}, score {score:.2f}")
        found_vertex = None
        
        while stack:
            score, current_vertex = heapq.heappop(stack)
            if current_vertex in visited:
                continue
            visited.add(current_vertex)

            # Check if this vertex is the closest to the target point
            if score <= min_score or (found_vertex is None):
                num_hops += 1
                score_text = Text(f"Visiting {current_vertex}, Score: {score:.2f}", font_size=24)
                score_text.to_corner(UL)
                self.play(AnimationGroup(Write(score_text), graph[current_vertex].animate.set_color(YELLOW)))
                self.wait(0.5)
                previous = found_vertex
                found_vertex = current_vertex
                min_score = score
                animations = []
                if previous is not None:
                    animations.append(graph[previous].animate.set_color(WHITE))
                animations.append(graph[current_vertex].animate.set_color(GREEN))
                animations.append(FadeOut(score_text))
                self.play(*animations)
                for neighbor in idx_to_neighbors.get(current_vertex, []):
                    neighbor_score = calculate_distance(threed_points[neighbor], target_point)
                    print(f"{current_vertex} Adding Neighbor {neighbor}, Score: {neighbor_score:.2f} Min Score: {min_score:.2f}")
                    if neighbor not in visited and neighbor_score < min_score:
                        heapq.heappush(stack, (neighbor_score, neighbor))


        # Highlight the found vertex
        final_text = Text(f"Found Nearest Neighbor in {num_hops - 1} hops!", font_size=36).to_edge(UP).set_color(GREEN)
        #self.add_fixed_in_frame_mobjects(final_text)
        self.play(Write(final_text))
        self.wait(2)

Second layer has 6 points
Total edges before adding second layer: [(100, 103), (100, 104), (101, 102), (101, 104), (102, 101), (102, 104), (103, 100), (103, 102), (104, 102), (104, 100), (105, 102), (105, 103)]
3D Graph has 106 points, starting at 104


Manim Community v0.18.1

second_layer_offset: 100
second_layer_vertices: [10, 29, 42, 58, 72, 98]
len(second_layer_vertices): 6
Visiting 104, Score: 5.48


getting neighbors for 104 in second layer: {100: [104, 103, 104], 103: [102, 105], 104: [100], 101: [104, 102], 102: [104, 103, 104, 105], 105: [103]}
104 Adding Neighbor 100, Score: 4.57 Min Score: 5.48
Visiting 100, Score: 4.57


getting neighbors for 100 in second layer: {100: [104, 103, 104], 103: [102, 105], 104: [100], 101: [104, 102], 102: [104, 103, 104, 105], 105: [103]}
100 Adding Neighbor 104, Score: 5.48 Min Score: 4.57
100 Adding Neighbor 103, Score: 4.48 Min Score: 4.57
100 Adding Neighbor 104, Score: 5.48 Min Score: 4.57
Visiting 103, Score: 4.48


getting neighbors for 103 in second layer: {100: [104, 103, 104], 103: [102, 105], 104: [100], 101: [104, 102], 102: [104, 103, 104, 105], 105: [103]}
103 Adding Neighbor 102, Score: 3.82 Min Score: 4.48
103 Adding Neighbor 105, Score: 0.86 Min Score: 4.48
Visiting 105, Score: 0.86


getting neighbors for 105 in second layer: {100: [104, 103, 104], 103: [102, 105], 104: [100], 101: [104, 102], 102: [104, 103, 104, 105], 105: [103]}
105 Adding Neighbor 103, Score: 4.48 Min Score: 0.86


init full graph search from 105, score 0.86
second_layer_vertex: 105 adjusted: 5
related first layer: 98
original_vertex_idx: 98
Starting full graph search from 98, score 0.86


98 Adding Neighbor 27, Score: 0.16 Min Score: 0.86
98 Adding Neighbor 57, Score: 0.35 Min Score: 0.86
98 Adding Neighbor 60, Score: 1.34 Min Score: 0.86
98 Adding Neighbor 63, Score: 1.43 Min Score: 0.86
98 Adding Neighbor 78, Score: 0.99 Min Score: 0.86
98 Adding Neighbor 60, Score: 1.34 Min Score: 0.86
98 Adding Neighbor 57, Score: 0.35 Min Score: 0.86
98 Adding Neighbor 63, Score: 1.43 Min Score: 0.86
98 Adding Neighbor 27, Score: 0.16 Min Score: 0.86
98 Adding Neighbor 78, Score: 0.99 Min Score: 0.86


27 Adding Neighbor 0, Score: 1.29 Min Score: 0.16
27 Adding Neighbor 57, Score: 0.35 Min Score: 0.16
27 Adding Neighbor 98, Score: 0.86 Min Score: 0.16
27 Adding Neighbor 78, Score: 0.99 Min Score: 0.16
27 Adding Neighbor 12, Score: 1.03 Min Score: 0.16
27 Adding Neighbor 60, Score: 1.34 Min Score: 0.16
27 Adding Neighbor 57, Score: 0.35 Min Score: 0.16
27 Adding Neighbor 69, Score: 1.39 Min Score: 0.16
27 Adding Neighbor 98, Score: 0.86 Min Score: 0.16
